In [19]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
import time

In [20]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver", options=chrome_options)

In [21]:
## Get target website as following ##
## We first connect Florida Department of Health website ##
chrome.get("https://appsmqa.doh.state.fl.us/MQASearchServices/HealthCareProviders")

In [22]:
def getPractitioners(board, profession, licenseStatus):
    ## Select Board Option as Board of Osteopathic Medicine ##
    chrome.find_element_by_xpath("//select[@name='SearchDto.Board']/option[text()='"+board+"']").click()
    
    time.sleep(1)
    
    ## Select Profession as Osteopathic Physician ##
    chrome.find_element_by_xpath("//select[@name='SearchDto.Profession']/option[text()='"+profession+"']").click()
    
    time.sleep(1)
    
    ## Select only Active/Clear doctors ##
    chrome.find_element_by_xpath("//select[@name='SearchDto.LicenseStatus']/option[@value='"+licenseStatus+"']").click()
    
    time.sleep(1)
    
    ## Click on Search button ## 
    chrome.find_element_by_xpath("//input[@value='Search']").click()
    
    urlList = []
    while(True):
        ## Finds all the links from the source ##
        content = chrome.page_source
        soup = BeautifulSoup(content)
        links = soup.find_all('a',href=True)
        for i in links:
            ## When navigate to one practitioner's profile, we see the following link common for each practitioner profile page
            link = "https://appsmqa.doh.state.fl.us/MQASearchServices/HealthCareProviders/LicenseVerification/Details?"
            if "LicInd" in i.get("href"):
                #print(i.get("href").split("?")[1])
                result = (i.get("href").split("?")[1]).index('&org')
                licind = ((i.get("href").split("?")[1])[:result])
                link += licind
                urlList.append(link)
        try:
            chrome.find_element_by_link_text("»").click()  
        except:
            break
    return urlList

In [ ]:
## Get Practitioners' Profile Links for the following profession and values ##

In [23]:
urlList_1 = getPractitioners("BOARD OF OSTEOPATHIC MEDICINE", "Osteopathic Physician", "ACT")

In [32]:
export_boom = pd.read_csv('Export_doh_boom.csv')

In [34]:
export_boom['Practitioner_profile'] = urlList_1

In [ ]:
## Get Practitioners' Profile Links for the following profession and values ##

In [ ]:
urlList_2 = getPractitioners("BOARD OF MEDICINE", "Medical Doctor", "ACT")

In [36]:
export_bom = pd.read_csv('Export_doh_bom.csv')

In [ ]:
export_bom['Practitioner_profile'] = urlList_2